In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_LLM_MODEL = os.getenv('OPENAI_LLM_MODEL')
OPENAI_EMBEDDING_MODEL = os.getenv('OPENAI_EMBEDDING_MODEL')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_INDEX_METRIC = os.getenv('PINECONE_INDEX_METRIC')
PINECONE_INDEX_DIMENSION = int(os.getenv('PINECONE_INDEX_DIMENSION'))
PINECONE_INDEX_REGION = os.getenv('PINECONE_INDEX_REGION')
PINECONE_INDEX_CLOUD = os.getenv('PINECONE_INDEX_CLOUD')

In [3]:
# pinecone 생성
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment = PINECONE_ENVIRONMENT
)
pc = Pinecone(api_key=PINECONE_API_KEY)
pc.create_index(
    name=PINECONE_INDEX_NAME,
    dimension=PINECONE_INDEX_DIMENSION,  # Dimension of the OpenAI embeddings
    metric=PINECONE_INDEX_METRIC,
  # Similarity metric
    spec=ServerlessSpec(
        cloud=PINECONE_INDEX_CLOUD,
        region=PINECONE_INDEX_REGION
    )
)


c:\Users\511-17\miniconda3\envs\langchain_part_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "name": "wine-reviews",
    "metric": "cosine",
    "host": "wine-reviews-bc4xat1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [6]:
wine_index = pc.Index(PINECONE_INDEX_NAME)
wine_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [12]:
from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(
    file_path='winemag-data-130k-v2.csv',
    encoding='utf-8'
)
docs = loader.load()
docs[0]

Document(metadata={'source': 'winemag-data-130k-v2.csv', 'row': 0}, page_content=": 0\ncountry: Italy\ndescription: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.\ndesignation: Vulkà Bianco\npoints: 87\nprice: \nprovince: Sicily & Sardinia\nregion_1: Etna\nregion_2: \ntaster_name: Kerin O’Keefe\ntaster_twitter_handle: @kerinokeefe\ntitle: Nicosia 2013 Vulkà Bianco  (Etna)\nvariety: White Blend\nwinery: Nicosia")

In [18]:
print(len(docs))
max([len(doc.page_content) for doc in docs])

129971


1115

In [19]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model = OPENAI_EMBEDDING_MODEL)

In [ ]:
from langchain_pinecone import PineconeVectorStore
BATCH_SIZE = 300
for i in range(0,len(docs), BATCH_SIZE):  # pinecone은 한번에 4mb 이상 처리 할수 없음, 그리고 임베딩 할때.. 임베딩 모델이 우리가 선택한게 8k 모델이라서.. 이런것들을 고려해야함
    try:
        batch = docs[i:i+BATCH_SIZE]
        vector_store = PineconeVectorStore.from_documents(
            index_name = PINECONE_INDEX_NAME,
            embedding=embedding,
            documents=batch,        
        )
        print(f'{i+1} : {len(batch)}') 
    except Exception as e:
        print(e)

1 : 300
301 : 300
601 : 300
901 : 300
1201 : 300
1501 : 300
1801 : 300
2101 : 300
2401 : 300
2701 : 300
3001 : 300
3301 : 300
3601 : 300
3901 : 300
4201 : 300
4501 : 300
4801 : 300
5101 : 300
5401 : 300
5701 : 300
6001 : 300
6301 : 300
6601 : 300
6901 : 300
7201 : 300
7501 : 300
7801 : 300
8101 : 300
8401 : 300
8701 : 300
9001 : 300
9301 : 300
9601 : 300
9901 : 300
10201 : 300
10501 : 300
10801 : 300
11101 : 300
11401 : 300
11701 : 300
12001 : 300
12301 : 300
12601 : 300
12901 : 300
13201 : 300
13501 : 300
13801 : 300
14101 : 300
14401 : 300
14701 : 300
15001 : 300
15301 : 300
15601 : 300
15901 : 300
16201 : 300
16501 : 300
16801 : 300
17101 : 300
17401 : 300
17701 : 300
18001 : 300
18301 : 300
18601 : 300
18901 : 300
19201 : 300
19501 : 300
19801 : 300
20101 : 300
20401 : 300
20701 : 300
21001 : 300
21301 : 300
21601 : 300
21901 : 300


In [25]:
range(len(docs))

range(0, 129971)